In [ ]:
# ENTER YOUR NAME AND ID HERE
# Sugandhi Akula - A20223858

# Project-2: Locality Sensitive Hashing

In [3]:
import random
import os
import numpy as np
import itertools
from random import randrange, shuffle
from shutil import copyfile
import collections

## Execute the follwing two cells to generate your data sample

In [4]:
MY_ID = '858'
# MY_ID should be string
# Example MY_ID='819'



try:
    n_1 = int(MY_ID[-1]) % 5
    n_2 = int(MY_ID[-2]) % 5
    n_3 = int(MY_ID[-3]) % 5
    
    while n_1 == n_2 or n_1 == n_3:
        n_1 = (n_1 + 1) % 5
    
    while n_1 == n_2 or n_2 == n_3:
        n_2 = (n_2 + 1) % 5
        
except Exception as e:
    print('Please enter a valid ID...')
    

task_dict = {0:'taska',1:'taskb',2:'taskc',3:'taskd',4:'taske'}
id_dict = {0:n_1,1:n_2,2:n_3}

In [5]:
os.chdir('/Users/sugandhi.akula/Desktop/CS 5683/Data/')

In [6]:
data_path = '/Users/sugandhi.akula/Desktop/CS 5683/Data/corpus-20090418' 
# Edit this path if the data directory is not in the current directory

try:
    os.makedirs('Data_Sample')
    os.makedirs('Original_Sample')
except Exception as e:
    pass

for i in range(3):
    task = task_dict[id_dict[i]]
    
    for file in os.listdir(data_path):
        if task in file:
            if 'orig' in file:
                copyfile(data_path + '/' + file, 'Original_Sample/' + file)
            else:
                copyfile(data_path + '/' + file, 'Data_Sample/' + file)

Your dataset for this project will be in <b>Data_Sample</b>


Your query choices will be in <b>Original_Sample</b> directory


You have to use any one original Wikipedia article from <b>Original_Sample</b> for <b>Fact Check</b> steps

### STEP - 1: Shingling (20 points)

In [7]:
# Type your code here... 
# Create necessary number of cells below this cell
k = 3

# NOTE: No complex text processing is required
# convert just upper case characters to lower case


Storing all sample documents' contents in a list of [(doc name, doc_contents)]:

In [8]:
os.chdir('/Users/sugandhi.akula/Desktop/CS 5683/Data/Data_Sample')
data_sample = []
for file in os.listdir():
    with open(file, encoding="utf8", errors='ignore') as f:
        data_sample.append((file,f.read().lower()))
    

Storing all original documents' contents in a list of [(doc name, doc_contents)]:

In [9]:
os.chdir('/Users/sugandhi.akula/Desktop/CS 5683/Data/Original_Sample')
original_sample = []
for file in os.listdir():
    with open(file, encoding="utf8", errors='ignore') as f:
        original_sample.append((file,f.read().lower()))

Cleaning the text; converting to lower case and removing all punctuations and special characters:

In [36]:
def clean_string(text):
    for ch in ['\n','*','_','{','}','[',']','(',')','<','>','#','+','-','.','!','$','\\',',','?','"','|','/','=']:
        if ch in text:
            text = text.replace(ch," ").strip()
    return text

In [37]:
clean_data_sample = [(x[0],clean_string(x[1])) for x in data_sample]

In [38]:
clean_original_sample = [(x[0],clean_string(x[1])) for x in original_sample]

In [39]:
clean_data_sample

[('g0pE_taskd.txt',
  'bayes theorem is an important theorem relating conditional probabilities  it allows us to calculate prob a b  from prob b a   bayes theorem is important because it can save us from gathering vast amounts of statistical evidence  the main theory is prob a b    prob b a    prob a   prob b   it means using prob win rain  from earlier  we can find the probability that it rained on a day that harry won a race'),
 ('g0pE_taske.txt',
  'dynamic programming is a method of solving problems that exhibit the properties of overlapping subproblems and optimal substructure  described below   the method takes much less time than naive methods  the word  programming  in  dynamic programming  has no particular connection to computer programming at all  and instead comes from the term  mathematical programming   a synonym for optimization  thus  the  program  is the optimal plan for action that is produced  for instance  a finalized schedule of events at an exhibition is sometimes

In [40]:
clean_original_sample

[('orig_taskd.txt',
  "in probability theory  bayes' theorem  often called bayes' law after rev thomas bayes  relates the conditional and marginal probabilities of two random events  it is often used to compute posterior probabilities given observations  for example  a patient may be observed to have certain symptoms  bayes' theorem can be used to compute the probability that a proposed diagnosis is correct  given that observation   see example 2  as a formal theorem  bayes' theorem is valid in all common interpretations of probability  however  it plays a central role in the debate around the foundations of statistics: frequentist and bayesian interpretations disagree about the ways in which probabilities should be assigned in applications  frequentists assign probabilities to random events according to their frequencies of occurrence or to subsets of populations as proportions of the whole  while bayesians describe probabilities in terms of beliefs and degrees of uncertainty  the art

Function for constructing shingles from the documents:

In [48]:
def get_shingles(size,doc):
    shingle_dict = {}
    for i in range (0,len(doc)):
        doc_shing = []
        lenth = len(doc[i][1])
        docid = doc[i][0]
        # Tokenizing text to aid in the formation of shingles
        tokens = doc[i][1].split()
        for j in range (0,lenth-size+1):
            sh = tuple(tokens[j:j+size])
            if sh not in doc_shing:
                doc_shing.append(sh)
        # Final shingle dictionary: {"docid":[shingles]}        
        shingle_dict[docid] = doc_shing
    return shingle_dict


Constructing 3-, 4- and 5-shingles and consolidating a list of these shingles from both the sample documents and the original documents:

In [42]:
shing_data_3 = set(list(itertools.chain(*list(get_shingles(3, clean_data_sample).values()))))
shing_orig_3 = set(list(itertools.chain(*list(get_shingles(3, clean_original_sample).values()))))
shing_3 = shing_data_3 | shing_orig_3
length_3 = len(shing_3)
# Total number of 3-shingles in both sample and original data files
length_3

7852

In [43]:
shing_data_4 = set(list(itertools.chain(*list(get_shingles(4, clean_data_sample).values()))))
shing_orig_4 = set(list(itertools.chain(*list(get_shingles(4, clean_original_sample).values()))))
shing_4 = shing_data_4 | shing_orig_4
length_4 = len(shing_4)
# Total number of 4-shingles in both sample and original data files
length_4

8786

In [44]:
shing_data_5 = set(list(itertools.chain(*list(get_shingles(5, clean_data_sample).values()))))
shing_orig_5 = set(list(itertools.chain(*list(get_shingles(5, clean_original_sample).values()))))
shing_5 = shing_data_5 | shing_orig_5
length_5 = len(shing_5)
# Total number of 5-shingles in both sample and original data files
length_5

9266

All sample documents and their constituent 5-shingles

In [210]:
shing_5_docs = get_shingles(5, clean_data_sample)
shing_5_docs = collections.OrderedDict(list(shing_5_docs.items()))
# Ordered dictionary for referencing documents and their shingles by index
shing_5_docs

OrderedDict([('g0pE_taskd.txt',
              [('bayes', 'theorem', 'is', 'an', 'important'),
               ('theorem', 'is', 'an', 'important', 'theorem'),
               ('is', 'an', 'important', 'theorem', 'relating'),
               ('an', 'important', 'theorem', 'relating', 'conditional'),
               ('important',
                'theorem',
                'relating',
                'conditional',
                'probabilities'),
               ('theorem', 'relating', 'conditional', 'probabilities', 'it'),
               ('relating', 'conditional', 'probabilities', 'it', 'allows'),
               ('conditional', 'probabilities', 'it', 'allows', 'us'),
               ('probabilities', 'it', 'allows', 'us', 'to'),
               ('it', 'allows', 'us', 'to', 'calculate'),
               ('allows', 'us', 'to', 'calculate', 'prob'),
               ('us', 'to', 'calculate', 'prob', 'a'),
               ('to', 'calculate', 'prob', 'a', 'b'),
               ('calculate', 'prob', 'a

In [52]:
# Type your code to get the 5-shingle index here


In [53]:
shing_5_list = list(shing_5)
shing_5_ind = {k: v for v, k in enumerate(shing_5_list)}
# Creating a dictionary of indexes for all 5-shingles
shing_5_ind

{('optimization', 'which', 'is', 'commonly', 'referred'): 0,
 ('given', 'a', "bayes'", 'theorem', 'relates'): 1,
 ('has', 'all', 'of', 'the', 'properties'): 2,
 ('the', 'colour', 'of', 'the', 'ball'): 3,
 ('bellman', 'originally', 'coined', 'the', 'term'): 4,
 ("'dynamic", "programming'", 'relates', 'to', 'the'): 5,
 ('or', 'that', 'bridges', 'and', 'skyscrapers'): 6,
 ('already', 'solved', 'subproblems'): 7,
 ('subproblems', 'these', 'subproblems', 'are', 'not'): 8,
 ('hierarchy', 'that', 'is', 'formed', 'is'): 9,
 ('girls', 'the', 'girls', 'wear', 'skirts'): 10,
 ('subproblems', 'are', 'not', 'independent', 'it'): 11,
 ('given', 'b', 'it', 'is', 'also'): 12,
 ('inheritance', 'from', 'several', 'classes'): 13,
 ('used', 'when', 'we', 'have', 'observations'): 14,
 ('in', "bayes'", 'theorem', 'are', 'named'): 15,
 ('known', 'to', 'have', 'a', 'further'): 16,
 ('also', 'be', 'termed', 'as', 'an'): 17,
 ('recent', 'years', 'dynamic', 'programming', 'languages'): 18,
 ('classes', 'of', 'ob

### STEP - 2: Min-Hashing (40 points)

In [54]:
# ***************************************************NEW***********************************************************
# Generate Hash functions - 
    # We use (ax + b) mod N formula to permute shingle index
    # where a,b are random numbers, N total index size, and x is the index
# We need to do L permutations - In other words we need to have L permutations (lists) of new indexes
# Following function takes total index size N and L as arguments
    # And returns L new lists of size N
    
def get_hash_functions(N,L):
    hash_functions = []
    
    for itr in range(L):
        a=randrange(1,400)
        b=randrange(1,400)
        
        new_hash_function = []
        for i in range(N):
            new_hash_function.append((a * i + b) % N)
        
        hash_functions.append(new_hash_function)
    return hash_functions
        
# test
# hash_functions = get_hash_functions(5000,50)

In [55]:
# Type your code here to generate all L hash functions
# Generate hash functions only for shingle index created for k=5

#L = {50,100,200,500,1000}
N = length_5
hash_func50 = get_hash_functions(N, 50)
hash_func100 = get_hash_functions(N, 100)
hash_func200 = get_hash_functions(N, 200)
hash_func500 = get_hash_functions(N, 500)
hash_func1000 = get_hash_functions(N, 1000)

Function for composing the signature matrix. Spits out the signature matrix in transposed form:

In [214]:
# Calculates minhash values for each document, over all hash documents, avoiding the need for an input matrix
# Parameters:
# 1. List of hash functions
# 2. Number of hash functions
# 3. Dictionary of 5-shingles from sample documents
# 4. Dictionary of indexes of all shingles
# Returns signature matrix of dimensions: rows(# of documents) x columns(# of hash functions used, minhash values)
def get_sig_m(hash_f, L, sh_5, sh_5_index):
    sig_m = []
    i = 0
    # Iterating through sample documents:
    for k in sh_5.values():
        sig_list = list(np.ones(L) * np.inf)
        # Iterating through all shingles:
        for sh, v in sh_5_index.items():
            # Where a shingle from the dictionary of indexes is found in the document, determine
            # the minshash value
            if sh in k:
                for j in range(L):
                    if hash_f[j][v] < sig_list[j]:
                        sig_list[j] = hash_f[j][v]
        i = i + 1
        sig_m.append(sig_list)
    return np.array(sig_m)

In [215]:
signature_50 = get_sig_m(hash_func50,50, shing_5_docs,shing_5_ind)
signature_100 = get_sig_m(hash_func100,100, shing_5_docs,shing_5_ind)
signature_200 = get_sig_m(hash_func200,200, shing_5_docs,shing_5_ind)
signature_500 = get_sig_m(hash_func500,500, shing_5_docs,shing_5_ind)
signature_1000 = get_sig_m(hash_func1000,1000,shing_5_docs,shing_5_ind)

In [ ]:
# Type your code here to get the final signature matrix S here


In [68]:
# Type your code here to do the fact check 
#      with any one query document in the 'Original_Sample' directory

original = 'Original_Sample/orig_taskc.txt'

# STEP-1: Generate 5-shingles 
    # (if any shingles are not present in your shingle index, simply ignore them)
    
    
    
# STEP-2: Generate signature vector from L hash functions



# STEP-3: Calculate Jaccard similarity of signature vector of orginal doc.
    # and all other documents 
    
t = 0.20


All original documents and their constituent 5-shingles

In [69]:
sh_5_orig = get_shingles(5, clean_original_sample)
sh_5_orig = collections.OrderedDict(sh_5_orig)
# Creating a dictionary of indexes for all 5-shingles
sh_5_orig

OrderedDict([('orig_taskd.txt',
              [('in', 'probability', 'theory', "bayes'", 'theorem'),
               ('probability', 'theory', "bayes'", 'theorem', 'often'),
               ('theory', "bayes'", 'theorem', 'often', 'called'),
               ("bayes'", 'theorem', 'often', 'called', "bayes'"),
               ('theorem', 'often', 'called', "bayes'", 'law'),
               ('often', 'called', "bayes'", 'law', 'after'),
               ('called', "bayes'", 'law', 'after', 'rev'),
               ("bayes'", 'law', 'after', 'rev', 'thomas'),
               ('law', 'after', 'rev', 'thomas', 'bayes'),
               ('after', 'rev', 'thomas', 'bayes', 'relates'),
               ('rev', 'thomas', 'bayes', 'relates', 'the'),
               ('thomas', 'bayes', 'relates', 'the', 'conditional'),
               ('bayes', 'relates', 'the', 'conditional', 'and'),
               ('relates', 'the', 'conditional', 'and', 'marginal'),
               ('the', 'conditional', 'and', 'marginal', 'pr

Calculating the jaccard distance: intersection/union

In [70]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

Function for caluclating the similarity of documents:

In [80]:
# Function that gauges similarity of documents by calculating the jaccard similarity of 
# sets of minhash values of documents
# Parameters:
# 1. (Transposed) signature matrix of sample documents
# 2. (Transposed) signature matrix of original documents
# 3. Threshold
# Returns a list of sample-original document pairs with a jaccard similarity greater than the supplied threshold

# List of the shingles from the sample documents and the original documents
da = list(shing_5_docs.items())
ori = list(sh_5_orig.items())
def sim_docs(sign_samp, sign_orig, t):
    sim = collections.defaultdict()
    # Calculating the jaccard similarities of all sample documents against all original documents
    for i in range(0,len(sign_samp)):
        for j in range(0,len(sign_orig)):
            card = jaccard(sign_samp[i].tolist(),sign_orig[j].tolist())
            # If the similarity score is greater than the threshold, output
            if card >= t:
                d = da[i][0]
                o = ori[j][0]
                k = d+" "+o
                sim[d+" & "+o] = card
    return sorted(sim.items(), key=lambda k_v: k_v[1], reverse=True)

Computing signature matrices for original documents with a varied number of hash functions:

In [216]:
signature_50_orig = get_sig_m(hash_func50,50,sh_5_orig,shing_5_ind)
signature_100_orig = get_sig_m(hash_func100,100,sh_5_orig,shing_5_ind)
signature_200_orig = get_sig_m(hash_func200,200,sh_5_orig,shing_5_ind)
signature_500_orig = get_sig_m(hash_func500,500,sh_5_orig,shing_5_ind)
signature_1000_orig = get_sig_m(hash_func1000,1000,sh_5_orig,shing_5_ind)

Documents that have a similarity greater than the threshold:

In [81]:
sim_docs(signature_50, signature_50_orig, 0.2)

[('g0pE_taska.txt & orig_taska.txt', 0.4925373134328358),
 ('g4pC_taska.txt & orig_taska.txt', 0.4492753623188406),
 ('g3pA_taskd.txt & orig_taskd.txt', 0.4492753623188406),
 ('g4pC_taskd.txt & orig_taskd.txt', 0.3888888888888889),
 ('g2pC_taska.txt & orig_taska.txt', 0.3333333333333333),
 ('g4pB_taske.txt & orig_taske.txt', 0.3333333333333333),
 ('g0pD_taska.txt & orig_taska.txt', 0.3157894736842105),
 ('g2pB_taske.txt & orig_taske.txt', 0.3157894736842105),
 ('g1pB_taske.txt & orig_taska.txt', 0.3157894736842105),
 ('g2pB_taskd.txt & orig_taskd.txt', 0.2987012987012987),
 ('g0pB_taske.txt & orig_taska.txt', 0.28205128205128205),
 ('g4pC_taske.txt & orig_taska.txt', 0.28205128205128205),
 ('g1pB_taske.txt & orig_taske.txt', 0.28205128205128205),
 ('g2pE_taska.txt & orig_taske.txt', 0.28205128205128205),
 ('g1pA_taskd.txt & orig_taskd.txt', 0.26582278481012656),
 ('g4pE_taskd.txt & orig_taska.txt', 0.26582278481012656),
 ('g2pB_taske.txt & orig_taska.txt', 0.26582278481012656),
 ('g4pC

In [83]:
sim_docs(signature_100, signature_100_orig, 0.2)

[('g0pE_taska.txt & orig_taska.txt', 0.3157894736842105),
 ('g4pC_taska.txt & orig_taska.txt', 0.2987012987012987),
 ('g3pA_taskd.txt & orig_taskd.txt', 0.2987012987012987),
 ('g0pB_taskd.txt & orig_taska.txt', 0.2578616352201258),
 ('g2pC_taska.txt & orig_taska.txt', 0.2578616352201258),
 ('g1pA_taskd.txt & orig_taskd.txt', 0.2422360248447205),
 ('g1pA_taskd.txt & orig_taska.txt', 0.2422360248447205),
 ('g0pD_taska.txt & orig_taska.txt', 0.2422360248447205),
 ('g2pE_taska.txt & orig_taska.txt', 0.2422360248447205),
 ('g1pA_taska.txt & orig_taska.txt', 0.2345679012345679),
 ('g2pA_taska.txt & orig_taska.txt', 0.2345679012345679),
 ('g0pB_taska.txt & orig_taska.txt', 0.2345679012345679),
 ('g2pB_taske.txt & orig_taske.txt', 0.2345679012345679),
 ('g2pB_taske.txt & orig_taska.txt', 0.2345679012345679),
 ('g4pC_taskd.txt & orig_taskd.txt', 0.2345679012345679),
 ('g4pC_taskd.txt & orig_taska.txt', 0.2345679012345679),
 ('g4pB_taskd.txt & orig_taska.txt', 0.2345679012345679),
 ('g4pB_taske.

In [84]:
sim_docs(signature_200, signature_200_orig, 0.2)

[('g3pA_taskd.txt & orig_taskd.txt', 0.23839009287925697),
 ('g0pE_taska.txt & orig_taska.txt', 0.21580547112462006),
 ('g4pC_taska.txt & orig_taska.txt', 0.21212121212121213),
 ('g1pB_taskd.txt & orig_taskd.txt', 0.20481927710843373),
 ('g4pB_taska.txt & orig_taskd.txt', 0.20481927710843373),
 ('g0pB_taskd.txt & orig_taskd.txt', 0.2012012012012012),
 ('g2pB_taskd.txt & orig_taskd.txt', 0.2012012012012012),
 ('g2pE_taskd.txt & orig_taskd.txt', 0.2012012012012012)]

In [86]:
sim_docs(signature_500, signature_500_orig, 0.1)

[('g3pA_taskd.txt & orig_taskd.txt', 0.10741971207087486),
 ('g3pC_taske.txt & orig_taskd.txt', 0.10497237569060773),
 ('g1pB_taskd.txt & orig_taskd.txt', 0.10497237569060773),
 ('g1pA_taske.txt & orig_taskd.txt', 0.10375275938189846),
 ('g3pB_taskd.txt & orig_taskd.txt', 0.10375275938189846),
 ('g0pD_taske.txt & orig_taskd.txt', 0.10375275938189846),
 ('g3pC_taska.txt & orig_taskd.txt', 0.10375275938189846),
 ('g0pC_taske.txt & orig_taskd.txt', 0.1013215859030837),
 ('g0pC_taskd.txt & orig_taskd.txt', 0.1013215859030837),
 ('g0pA_taske.txt & orig_taskd.txt', 0.1013215859030837),
 ('g2pE_taskd.txt & orig_taskd.txt', 0.1013215859030837),
 ('g2pC_taska.txt & orig_taskd.txt', 0.1013215859030837),
 ('g4pD_taska.txt & orig_taskd.txt', 0.1013215859030837),
 ('g0pE_taskd.txt & orig_taskd.txt', 0.1001100110011001),
 ('g0pB_taskd.txt & orig_taskd.txt', 0.1001100110011001),
 ('g4pC_taskd.txt & orig_taskd.txt', 0.1001100110011001),
 ('g1pB_taska.txt & orig_taskd.txt', 0.1001100110011001),
 ('g3pA

In [87]:
sim_docs(signature_1000, signature_1000_orig, 0.05)

[('g0pC_taskd.txt & orig_taskd.txt', 0.0672358591248666),
 ('g0pD_taske.txt & orig_taskd.txt', 0.0672358591248666),
 ('g3pA_taskd.txt & orig_taskd.txt', 0.0672358591248666),
 ('g0pE_taskd.txt & orig_taskd.txt', 0.06666666666666667),
 ('g3pB_taske.txt & orig_taskd.txt', 0.06666666666666667),
 ('g3pC_taska.txt & orig_taskd.txt', 0.06666666666666667),
 ('g0pE_taske.txt & orig_taskd.txt', 0.06609808102345416),
 ('g3pB_taskd.txt & orig_taskd.txt', 0.06609808102345416),
 ('g3pC_taskd.txt & orig_taskd.txt', 0.06609808102345416),
 ('g3pC_taske.txt & orig_taskd.txt', 0.06609808102345416),
 ('g1pD_taskd.txt & orig_taskd.txt', 0.06609808102345416),
 ('g0pC_taska.txt & orig_taskd.txt', 0.06553010122535961),
 ('g2pE_taskd.txt & orig_taskd.txt', 0.06553010122535961),
 ('g0pD_taskd.txt & orig_taskd.txt', 0.06496272630457935),
 ('g4pD_taska.txt & orig_taskd.txt', 0.06439595529536987),
 ('g0pC_taske.txt & orig_taskd.txt', 0.06382978723404255),
 ('g1pA_taska.txt & orig_taskd.txt', 0.06382978723404255),


### STEP - 3: LSH (30 points)

In [88]:
# Type your code here to hash signature matrix into B buckets
# Use the technique to split the signature matrix into b bands of r rows
# Convert only the signature matrix generated with L=1000

b = 50
r = 20
B = 199


Segregating the signature function into b bands of r rows each:

In [89]:
# Function takes the signature matrix in transposed form, and splits every document's 
# minhash values into b bands of r rows
# Parameters:
# 1. Signature matrix in transposed form
# 2. Number of bands
# 3. Number of rows in each band
# Returns a matrix of separated bands
def bands(sig,b,r):
    l = len(sig[0])
    x_banded = []
    for doc in sig:
        x_bands = []
        n = 0
        while n <= l-r:
            x_bands.append(tuple(doc[n:n+r]))
            n = n+r
        x_banded.append(x_bands)
    return np.array(x_banded)

Function to hash the values in the b bands:

In [225]:
# Function hashes the minhashes values using the formula: (sum over band(minhash value*random integer a)) % N
# Parameters:
# 1. Banded matrix from previous step
# 2. Number of buckets, N
def hash_vals(band_mat, N):
    # Generating a list of random integers between 1 and 20 that will be used across bands
    a = list(range(1,r+1))
    shuffle(a)
    lsh = []
    for doc in band_mat:
        hashed = []
        for v in doc:
            # Hashing the minhash values into buckets
            summed = sum([v[x]*a[x]for x in range(r)])%N
            hashed.append(summed)
        lsh.append(hashed)
    return np.array(lsh)

In [ ]:
# Type your code here to do generate candidate documents
# Follow all steps from STEP - 2 fact check (except the Jaccard similarity part)

# STEP - 1: Split your original document signature vector into b bands of r rows

# STEP - 2: Hash using the same hash functions created for 
    # signature matrix hashing (in the previous cell)
    

Banding and hashing the sample documents' and original documents' minhash signatures:

In [200]:
banded_mat = bands(signature_1000,b,r)
hashed_docs = hash_vals(banded_mat,B)
banded_mat_orig = bands(signature_1000_orig,b,r)
hashed_docs_orig = hash_vals(banded_mat_orig,B)
# One document from the set of original documents available
test_orig = hashed_docs_orig[1].tolist()
# Name of the aforementioned original document
orig_name = list(sh_5_orig.items())[1][0]

In [201]:
# Type your code here to do the fact check
# Calculate Jaccard similarity of the roiginal document with only 
    # candidate documents
    

Function for Locality Sensitive Hashing:

In [209]:
# This function takes the banded and hashed matrices for the sample and original documents and 
# computes similarity based on the principal that similar documents are likely to end up in
# the same bucket
# Parameters:
# 1. Banded + hashed sample documents
# 2. Dictionary of sample documents and constiuent 5-shingles
# 3. Original banded + hashed document
# 4. Threshold
# Returns:
# 1. Candidates (at least one same hash value shared between sample document and original document)
# 2. False negatives
# 3. False positives
# 4. Jaccard similarity scores of candidate documents
def local_sh(hash_docs, sh_5, test_o,t):
    cand = []
    jaccard_s = []
    false_neg = []
    false_pos = []
    ind = 0
    for x in hash_docs:
        fl = 0
        # Name of the sample document
        doc = list(sh_5.items())[ind][0]
        # A document is considered a candidate only when at least one of its banded+hashed value
        # is the shared with the banded+hashed values of the original document
        if len([i for i, j in zip(x, test_o) if i == j]) >= 1:
            cand.append(doc)
            fl = 1
        j = jaccard(list(x),test_o)
        if fl > 0:
            # Jaccard similarity scores of all candidate documents
            jaccard_s.append((doc,orig_name,j))
        # False negatives are those documents whose similarity scores are greater than the
        # threshold, but are not indentified as candidates
        if j > t and doc not in cand:
            false_neg.append(doc)
        # False positives are those documents whose similarity scores are lower than the
        # threshold, but are indentified as candidates
        elif j <= t and doc in cand:
            false_pos.append(doc)
        ind = ind+1
    jaccard_s.sort(key=lambda x:x[2], reverse=True)
    return (cand, false_neg, false_pos, jaccard_s)


In [203]:
candidates, false_negatives, false_positives, jaccards = local_sh(hashed_docs,shing_5_docs,test_orig,0.2)

In [204]:
candidates

['g0pE_taskd.txt',
 'g0pC_taska.txt',
 'g1pA_taskd.txt',
 'g0pC_taskd.txt',
 'g2pA_taska.txt',
 'g2pA_taske.txt',
 'g0pD_taska.txt',
 'g4pE_taske.txt',
 'g0pA_taske.txt',
 'g4pB_taska.txt',
 'g3pA_taskd.txt',
 'g2pC_taske.txt']

In [208]:
len(candidates)

12

In [205]:
jaccards

[('g3pA_taskd.txt', 'orig_taske.txt', 0.17647058823529413),
 ('g2pA_taske.txt', 'orig_taske.txt', 0.14942528735632185),
 ('g0pE_taskd.txt', 'orig_taske.txt', 0.13636363636363635),
 ('g0pD_taska.txt', 'orig_taske.txt', 0.13636363636363635),
 ('g4pE_taske.txt', 'orig_taske.txt', 0.13636363636363635),
 ('g0pC_taska.txt', 'orig_taske.txt', 0.12359550561797752),
 ('g0pC_taskd.txt', 'orig_taske.txt', 0.12359550561797752),
 ('g2pA_taska.txt', 'orig_taske.txt', 0.12359550561797752),
 ('g0pA_taske.txt', 'orig_taske.txt', 0.12359550561797752),
 ('g4pB_taska.txt', 'orig_taske.txt', 0.12359550561797752),
 ('g2pC_taske.txt', 'orig_taske.txt', 0.0989010989010989),
 ('g1pA_taskd.txt', 'orig_taske.txt', 0.08695652173913043)]

In [206]:
false_positives

['g0pE_taskd.txt',
 'g0pC_taska.txt',
 'g1pA_taskd.txt',
 'g0pC_taskd.txt',
 'g2pA_taska.txt',
 'g2pA_taske.txt',
 'g0pD_taska.txt',
 'g4pE_taske.txt',
 'g0pA_taske.txt',
 'g4pB_taska.txt',
 'g3pA_taskd.txt',
 'g2pC_taske.txt']

In [217]:
len(false_positives)

12

In [207]:
false_negatives

['g1pD_taska.txt']

In [218]:
len(false_negatives)

1

### BONUS : 10 points

In [ ]:
# Experiment with different values of b,r,B, and t
    # to reduce the number of false positives and false negatives
    # Report all results in a table in a separate word document

In [221]:
b = 10
r = 100
B = 200
t = 0.2
banded_mat = bands(signature_1000,b,r)
hashed_docs = hash_vals(banded_mat,B)
banded_mat_orig = bands(signature_1000_orig,b,r)
hashed_docs_orig = hash_vals(banded_mat_orig,B)
test_orig = hashed_docs_orig[1].tolist()
orig_name = list(sh_5_orig.items())[1][0]

candidates, false_negatives, false_positives, jaccards = local_sh(hashed_docs,shing_5_docs,test_orig,t)
len_candidates = len(candidates)
len_fn = len(false_negatives)
len_fp = len(false_positives)
print(jaccards)
print("Candidates:",len_candidates,", False negatives:",len_fn,", False positives:",len_fp)

[('g3pC_taska.txt', 'orig_taske.txt', 0.1111111111111111), ('g4pB_taska.txt', 'orig_taske.txt', 0.1111111111111111), ('g4pE_taska.txt', 'orig_taske.txt', 0.05263157894736842), ('g3pA_taske.txt', 'orig_taske.txt', 0.05263157894736842)]
Candidates: 4 , False negatives: 0 , False positives: 4


In [226]:
b = 100
r = 10
B = 250
t = 0.2
banded_mat = bands(signature_1000,b,r)
hashed_docs = hash_vals(banded_mat,B)
banded_mat_orig = bands(signature_1000_orig,b,r)
hashed_docs_orig = hash_vals(banded_mat_orig,B)
test_orig = hashed_docs_orig[1].tolist()
orig_name = list(sh_5_orig.items())[1][0]

candidates, false_negatives, false_positives, jaccards = local_sh(hashed_docs,shing_5_docs,test_orig,t)
len_candidates = len(candidates)
len_fn = len(false_negatives)
len_fp = len(false_positives)
print(jaccards)
print("Candidates:",len_candidates,", False negatives:",len_fn,", False positives:",len_fp)

[('g1pA_taske.txt', 'orig_taske.txt', 0.21212121212121213), ('g1pB_taska.txt', 'orig_taske.txt', 0.21212121212121213), ('g2pA_taskd.txt', 'orig_taske.txt', 0.19760479041916168), ('g2pB_taska.txt', 'orig_taske.txt', 0.19760479041916168), ('g2pC_taske.txt', 'orig_taske.txt', 0.1834319526627219), ('g4pC_taska.txt', 'orig_taske.txt', 0.1695906432748538), ('g0pA_taske.txt', 'orig_taske.txt', 0.1695906432748538), ('g4pB_taske.txt', 'orig_taske.txt', 0.1695906432748538), ('g2pE_taskd.txt', 'orig_taske.txt', 0.16279069767441862), ('g3pA_taskd.txt', 'orig_taske.txt', 0.16279069767441862), ('g0pD_taske.txt', 'orig_taske.txt', 0.15606936416184972), ('g4pB_taskd.txt', 'orig_taske.txt', 0.14942528735632185), ('g4pE_taskd.txt', 'orig_taske.txt', 0.13636363636363635), ('g4pC_taskd.txt', 'orig_taske.txt', 0.13636363636363635)]
Candidates: 14 , False negatives: 1 , False positives: 12


In [234]:
b = 2
r = 500
B = 500
t = 0.3
banded_mat = bands(signature_1000,b,r)
hashed_docs = hash_vals(banded_mat,B)
banded_mat_orig = bands(signature_1000_orig,b,r)
hashed_docs_orig = hash_vals(banded_mat_orig,B)
test_orig = hashed_docs_orig[1].tolist()
orig_name = list(sh_5_orig.items())[1][0]

candidates, false_negatives, false_positives, jaccards = local_sh(hashed_docs,shing_5_docs,test_orig,t)
len_candidates = len(candidates)
len_fn = len(false_negatives)
len_fp = len(false_positives)
print(jaccards)
print("Candidates:",len_candidates,", False negatives:",len_fn,", False positives:",len_fp)

[('g0pA_taska.txt', 'orig_taske.txt', 0.3333333333333333), ('g4pB_taske.txt', 'orig_taske.txt', 0.3333333333333333)]
Candidates: 2 , False negatives: 0 , False positives: 0


In [257]:
b = 40
r = 25
B = 200
t = 0.1
banded_mat = bands(signature_1000,b,r)
hashed_docs = hash_vals(banded_mat,B)
banded_mat_orig = bands(signature_1000_orig,b,r)
hashed_docs_orig = hash_vals(banded_mat_orig,B)
test_orig = hashed_docs_orig[1].tolist()
orig_name = list(sh_5_orig.items())[1][0]

candidates, false_negatives, false_positives, jaccards = local_sh(hashed_docs,shing_5_docs,test_orig,t)
len_candidates = len(candidates)
len_fn = len(false_negatives)
len_fp = len(false_positives)
#print(jaccards)
print("Candidates:",len_candidates,", False negatives:",len_fn,", False positives:",len_fp)

Candidates: 11 , False negatives: 25 , False positives: 7
